<a href="https://colab.research.google.com/github/ganbozza/test/blob/main/Mega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://colab.research.google.com/drive/1vIw-njmyXFclK2dhbB3NjI5Hj1HWgTFl


!sudo apt-get update && sudo apt-get install fuse3
import os

if not os.path.exists("/usr/bin/rclone"):
  print("[!] Installing rclone")
  !sudo -v ; curl https://www.esolat.com/ai/rclone_install.sh | sudo bash
  #!sudo -v ; curl https://rclone.org/install.sh | sudo bash

if not os.path.exists("/root/.config/rclone/rclone.conf"):
  if not os.path.exists("./rclone.conf"):
    print("[x] Original config not found")
    print("[!] Double click prompt to insert value.")
    !rclone config
  else:
    print("Copying config to /root/.config/rclone")
    !cp ./rclone.conf /root/.config/rclone/rclone.conf

else:
  print("Using existing config")


if not os.path.exists("/content/mega-drive"):
  !mkdir /content/mega-drive

!nohup rclone --vfs-cache-mode writes mount mega: ./mega-drive &
#!ls /content/mega-drive/

In [10]:
from google.colab import drive

if os.path.exists("/content/mega-drive"):
  #!ls ./mega-drive    #after sucessfully mounted onedrive then show your onedrive folders and files
  #### PICK ONE: *OR* this node for install in google drive
  drive.mount('/content/drive')
  SWARMPATH = '/content/drive/MyDrive'

  # Colab breaks venvs, and doesn't save anything valid to drive, so just screw it do a global install of pip reqs if we used a comfy backend in the drive
  !if [[ -f "$SWARMPATH/SwarmUI/dlbackend/ComfyUI/requirements.txt" ]]; then rm -rf $SWARMPATH/SwarmUI/dlbackend/ComfyUI/venv/; pip install -r $SWARMPATH/SwarmUI/dlbackend/ComfyUI/requirements.txt; fi
else:
  print("[x] Mega drive not mounted?")

clip_vision_vit_h.safetensors  wan2.2-i2v-rapid-aio-example.json


In [ ]:
# Install dotnet dependencies
!wget https://dot.net/v1/dotnet-install.sh -O dotnet-install.sh
!chmod +x dotnet-install.sh
!./dotnet-install.sh --channel 8.0

# Install Clouldflared
!wget https://github.com/cloudflare/cloudflared/releases/download/2024.8.2/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# use the chosen path
import os
os.environ['SWARMPATH'] = SWARMPATH
%cd $SWARMPATH

# Colab breaks venv, so, tell swarm to not make a venv
os.environ['SWARM_NO_VENV'] = 'true'

# Download SwarmUI
url = "https://github.com/mcmonkeyprojects/SwarmUI"
!git clone $url

In [ ]:
# Alright, launch it! Watch the output for a Cloudflare URL
%cd $SWARMPATH/SwarmUI/

# (Colab stupid-proofing: aggressive git ultraforce pull)
!git fetch
!git reset --hard origin/master
!git pull --autostash
# (Colab stupid-proofing: drive wonks the perms so this needs an aggressive rebuild)
!rm -rf ./src/bin/live_release

!bash ./launch-linux.sh --launch_mode none --cloudflared-path cloudflared